In [16]:
import pandas as pd
import numpy as np

## Problem 1
#### Find all unique employee names who work in more than one department

In [17]:
df1 = pd.DataFrame({'EmployeeName': ['John Doe', 'Jane Smith', 'Alice Johnson', 'John Doe'], 'Department': ['Sales', 'Marketing', 'Sales', 'Marketing']})

In [18]:
df1

,EmployeeName,Department
0,John Doe,Sales
1,Jane Smith,Marketing
2,Alice Johnson,Sales
3,John Doe,Marketing


### Approach 1 using transform function

In [19]:
df1[df1.groupby('EmployeeName')['Department'].transform('nunique')>1]['EmployeeName'].drop_duplicates()

0    John Doe
Name: EmployeeName, dtype: object

### Approach 2 using filter function

In [36]:
agg_df = df1.groupby('EmployeeName')
agg_df.filter(lambda x: x['Department'].nunique()>1)['EmployeeName'].drop_duplicates().tolist()

['John Doe']

### Approach 3 using boolean filter

In [41]:
df2=df1.groupby('EmployeeName').nunique()
df2[df2.Department>1].index.tolist()

['John Doe']

## Problem 2
#### Calculate the monthly average sales for each product. Assume sales data is daily

In [54]:
pr2 = pd.DataFrame({'Date': pd.date_range(start='2023-01-01', end='2023-03-31', freq='D'), 'Product': np.random.choice(['ProductA', 'ProductB'], 90), 'Sales': np.random.randint(100, 500, 90)})

In [57]:
pr2['month']=pr2['Date'].dt.month
pr2

,Date,Product,Sales,month
0,2023-01-01,ProductB,232,1
1,2023-01-02,ProductB,150,1
2,2023-01-03,ProductB,329,1
3,2023-01-04,ProductB,354,1
4,2023-01-05,ProductB,433,1
...,...,...,...,...
85,2023-03-27,ProductA,226,3
86,2023-03-28,ProductA,159,3
87,2023-03-29,ProductA,215,3
88,2023-03-30,ProductA,427,3


### Approach: Using agg function

In [58]:
pr2.groupby(['month','Product']).agg(average_sales=('Sales','mean')).round(2)

average_sales
month Product                
1     ProductA         289.37
      ProductB         304.08
2     ProductA         308.73
      ProductB         332.92
3     ProductA         284.00
      ProductB         292.64

## Problem 3
#### Identify the top 3 employees with the highest sales in each quarter

In [74]:
pr3 = pd.DataFrame({'Employee': ['John', 'Jane', 'Doe', 'Smith', 'Alice','Smith', 'Alice'], 'Quarter': ['Q1', 'Q1', 'Q2', 'Q2', 'Q3','Q1', 'Q1'], 'Sales': [200, 150, 300, 250, 400,300,500]})
pr3

,Employee,Quarter,Sales
0,John,Q1,200
1,Jane,Q1,150
2,Doe,Q2,300
3,Smith,Q2,250
4,Alice,Q3,400
5,Smith,Q1,300
6,Alice,Q1,500


### Approach 1 using groupby and rank

In [165]:
pr3['Rank']=pr3.groupby('Quarter')['Sales'].rank(ascending=False).astype('int')

In [79]:
pr3['Rank']=pr3.sort_values('Sales',ascending=False).groupby('Quarter')['Sales'].cumcount()+1
pr3

,Employee,Quarter,Sales,Rank
0,John,Q1,200,3
1,Jane,Q1,150,4
2,Doe,Q2,300,1
3,Smith,Q2,250,2
4,Alice,Q3,400,1
5,Smith,Q1,300,2
6,Alice,Q1,500,1


In [76]:
pr3[pr3['Rank']<4][['Employee','Quarter','Sales']]

,Employee,Quarter,Sales
0,John,Q1,200
2,Doe,Q2,300
3,Smith,Q2,250
4,Alice,Q3,400
5,Smith,Q1,300
6,Alice,Q1,500


### Approach 2 using groupby and nlargest

In [77]:
pr3.drop('Rank',axis=1,inplace=True)
pr3.groupby('Quarter').apply(lambda x: x.nlargest(3,'Sales'))

Employee Quarter  Sales
Quarter                          
Q1      6    Alice      Q1    500
        5    Smith      Q1    300
        0     John      Q1    200
Q2      2      Doe      Q2    300
        3    Smith      Q2    250
Q3      4    Alice      Q3    400

## Problem 4
#### Analyze the attendance records to find employees with more than 95% attendance throughout the year.

In [83]:
pr4 = pd.DataFrame({'Employee': ['John', 'Jane', 'Doe'], 'TotalDays': [365, 365, 365], 'DaysAttended': [365, 300, 360]})
pr4

,Employee,TotalDays,DaysAttended
0,John,365,365
1,Jane,365,300
2,Doe,365,360


In [86]:
pr4[pr4['DaysAttended']/pr4['TotalDays']>.95]['Employee'].tolist()

['John', 'Doe']

## Problem 5
#### Calculate the monthly customer retention rate based on the transaction logs.

In [87]:
pr5 = pd.DataFrame({'Month': ['Jan', 'Feb', 'Mar', 'Jan', 'Feb', 'Mar'], 'CustomerID': [1, 1, 1, 2, 2, 3], 'TransactionCount': [1, 2, 1, 3, 2, 1]})
pr5

,Month,CustomerID,TransactionCount
0,Jan,1,1
1,Feb,1,2
2,Mar,1,1
3,Jan,2,3
4,Feb,2,2
5,Mar,3,1


In [91]:
# Create a month num column
month_dict={'Jan':1,'Feb':2,'Mar':3}
pr5['MonthNum']=pr5['Month'].apply(lambda x: month_dict[x])

In [111]:
# calculate number of unique customers in each month
month_cust_df=pr5.groupby(['Month','MonthNum']).agg(month_cust=('CustomerID','nunique')).reset_index().sort_values('MonthNum')
month_cust_df

,Month,MonthNum,month_cust
1,Jan,1,2
0,Feb,2,2
2,Mar,3,2


In [94]:
# Create a dataframe of month and month num values
pr5_mon=pr5[['Month','MonthNum']].drop_duplicates()
pr5_mon

,Month,MonthNum
0,Jan,1
1,Feb,2
2,Mar,3


In [98]:
# Cross join month df and pr5 df
merge_df=pr5_mon.merge(pr5,how='cross')
merge_df

,Month_x,MonthNum_x,Month_y,CustomerID,TransactionCount,MonthNum_y
0,Jan,1,Jan,1,1,1
1,Jan,1,Feb,1,2,2
2,Jan,1,Mar,1,1,3
3,Jan,1,Jan,2,3,1
4,Jan,1,Feb,2,2,2
5,Jan,1,Mar,3,1,3
6,Feb,2,Jan,1,1,1
7,Feb,2,Feb,1,2,2
8,Feb,2,Mar,1,1,3
9,Feb,2,Jan,2,3,1


In [101]:
# filtering the conditions, so that the each month joins with either the same month or previous months
# Essentially, this is similar to inner join with condition "join on a.monthnum>=b.monthnum"
fil_df=merge_df.loc[merge_df['MonthNum_x']>=merge_df['MonthNum_y']]
fil_df

,Month_x,MonthNum_x,Month_y,CustomerID,TransactionCount,MonthNum_y
0,Jan,1,Jan,1,1,1
3,Jan,1,Jan,2,3,1
6,Feb,2,Jan,1,1,1
7,Feb,2,Feb,1,2,2
9,Feb,2,Jan,2,3,1
10,Feb,2,Feb,2,2,2
12,Mar,3,Jan,1,1,1
13,Mar,3,Feb,1,2,2
14,Mar,3,Mar,1,1,3
15,Mar,3,Jan,2,3,1


In [112]:
# finding cumulative distinct users for each month
cum_cust_df=fil_df.groupby(['Month_x','MonthNum_x']).agg(cum_cust=('CustomerID','nunique')).reset_index().sort_values('MonthNum_x')
cum_cust_df

,Month_x,MonthNum_x,cum_cust
1,Jan,1,2
0,Feb,2,2
2,Mar,3,3


In [113]:
cum_cust_df['Month_cust']=month_cust_df['month_cust']
cum_cust_df

,Month_x,MonthNum_x,cum_cust,Month_cust
1,Jan,1,2,2
0,Feb,2,2,2
2,Mar,3,3,2


In [121]:
# Calculating retention rate
cum_cust_df['retention_rate']=(cum_cust_df['Month_cust']/cum_cust_df['cum_cust']*100).round(2)

In [123]:
cum_cust_df.columns=['Month','MonthNum','cum_cust','Month_cust','retention_rate']

In [124]:
result=cum_cust_df[['Month','retention_rate']]
result

,Month,retention_rate
1,Jan,100.00
0,Feb,100.00
2,Mar,66.67


## Problem 6
#### Determine the average time employees spent on projects, assuming you have start and end dates for each project participation.

In [125]:
pr6 = pd.DataFrame({'Employee': ['John', 'Jane', 'Doe'], 'ProjectStart': pd.to_datetime(['2023-01-01', '2023-02-15', '2023-03-01']), 'ProjectEnd': pd.to_datetime(['2023-01-31', '2023-03-15', '2023-04-01'])})
pr6

,Employee,ProjectStart,ProjectEnd
0,John,2023-01-01,2023-01-31
1,Jane,2023-02-15,2023-03-15
2,Doe,2023-03-01,2023-04-01


In [129]:
pr6['duration']=(pr6['ProjectEnd']-pr6['ProjectStart']).dt.days
pr6['duration'].mean()

29.666666666666668

## Problem 7
#### Compute the month-on-month growth rate in sales for each product, highlighting products with more than 10% growth for consecutive months.

In [130]:
pr7 = pd.DataFrame({'Month': ['Jan', 'Feb', 'Mar', 'Jan', 'Feb', 'Mar'], 'Product': ['A', 'A', 'A', 'B', 'B', 'B'], 'Sales': [200, 220, 240, 150, 165, 180]})
pr7

,Month,Product,Sales
0,Jan,A,200
1,Feb,A,220
2,Mar,A,240
3,Jan,B,150
4,Feb,B,165
5,Mar,B,180


### Approach 1: Using pct_change function

In [142]:
pr7['growth']=pr7.groupby('Product')['Sales'].pct_change()*100
pr7['growth']=pr7.growth.round(2)
pr7

,Month,Product,Sales,growth
0,Jan,A,200,NaN
1,Feb,A,220,10.00
2,Mar,A,240,9.09
3,Jan,B,150,NaN
4,Feb,B,165,10.00
5,Mar,B,180,9.09


### Approach 2: Using shift function
Combine first function serves the similar function as coalesce function in SQL

In [149]:
pr7.drop('growth',axis=1,inplace=True)

In [151]:
pr7['prev_sales']=pr7.groupby('Product')['Sales'].shift(1)
pr7['prev_sales']=pr7['prev_sales'].combine_first(pr7['Sales'])
pr7['growth']=(pr7['Sales']-pr7['prev_sales'])*100/pr7['prev_sales']
pr7

,Month,Product,Sales,prev_sales,growth
0,Jan,A,200,200.0,0.000000
1,Feb,A,220,200.0,10.000000
2,Mar,A,240,220.0,9.090909
3,Jan,B,150,150.0,0.000000
4,Feb,B,165,150.0,10.000000
5,Mar,B,180,165.0,9.090909


## Problem 8
#### Identify the time of day (morning, afternoon, evening) when sales peak for each category of products.

In [152]:
pr8=pd.DataFrame({'Category': ['Electronics', 'Clothing', 'Electronics', 'Clothing'], 'TimeOfDay': ['Morning', 'Afternoon', 'Evening', 'Morning'], 'Sales': [300, 150, 500, 200]})
pr8

,Category,TimeOfDay,Sales
0,Electronics,Morning,300
1,Clothing,Afternoon,150
2,Electronics,Evening,500
3,Clothing,Morning,200


In [157]:
pr8['rnk']=pr8.groupby('Category')['Sales'].rank(ascending=False).astype('int')
pr8[pr8['rnk']==1][['Category','TimeOfDay']]

,Category,TimeOfDay
2,Electronics,Evening
3,Clothing,Morning


## Problem 9
#### Evaluate the distribution of workload among employees based on the number of tasks assigned in the last six months

In [158]:
pr9=pd.DataFrame({'Employee': ['John', 'Jane', 'Doe'], 'TasksAssigned': [20, 25, 15]})

In [159]:
pr9

,Employee,TasksAssigned
0,John,20
1,Jane,25
2,Doe,15


In [160]:
pr9.describe()

,TasksAssigned
count,3.0
mean,20.0
std,5.0
min,15.0
25%,17.5
50%,20.0
75%,22.5
max,25.0


## Problem 10
#### Calculate the profit margin for each product category based on revenue and cost data.

In [161]:
pr10=pd.DataFrame({'Category': ['Electronics', 'Clothing'], 'Revenue': [1000, 500], 'Cost': [700, 300]})    

In [162]:
pr10

,Category,Revenue,Cost
0,Electronics,1000,700
1,Clothing,500,300


In [164]:
pr10['margin']=(pr10['Revenue']-pr10['Cost'])*100/pr10['Revenue']
pr10[['Category','margin']]

,Category,margin
0,Electronics,30.0
1,Clothing,40.0
